In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from career_chatbot.state.state import State
from career_chatbot.nodes.rag_chatbot_node import RAGChatbotNode  # Adjust path if needed
from langchain_core.documents import Document

# Load environment variables
load_dotenv()
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# Initialize LLM
llm_model = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)

# Initialize RAGChatbotNode
rag_node = RAGChatbotNode(model=llm_model)

# Initial state
state = {"question": "What is weahter in Kondal province, Kingdom of Cambodia?", "documents": []}

def print_docs(docs, preview=100):
    if isinstance(docs, list):
        for i, doc in enumerate(docs):
            print(f"Document {i+1}:", doc.page_content[:preview], "...")
    elif isinstance(docs, Document):
        print(docs.page_content[:preview], "...")
    else:
        print(docs)

print("\n========== DEBUG SEQUENCE START ==========")

# 1. Transform Query
print("\n--- Step 1: transform_query ---")
state = rag_node.transform_query(state)
print("Transformed Question:", state["question"])

# 2. Route Question
print("\n--- Step 2: route_question ---")
route = rag_node.route_question(state)
print("Routed to:", route)

USER_AGENT environment variable not set, consider setting it to identify your requests.



========== DEBUG SEQUENCE START ==========

--- Step 1: transform_query ---
---TRANSORM QUERY---
Transformed Question: What is the current weather in Kandal province, Cambodia?

--- Step 2: route_question ---
---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
Routed to: web_search


In [2]:
# 3. Retrieve Documents
if route == "vector_store":
    print("\n--- Step 3: retrieve ---")
    state = rag_node.retrieve(state)
    print("Retrieved Documents:")
    print_docs(state["documents"])
elif route == "web_search":
    print("\n--- Step 3: web_search ---")
    state = rag_node.web_search(state)
    print("Web Search Documents:")
    print_docs(state["documents"])


--- Step 3: web_search ---
---WEB SEARCH---
Raw web search result: {'query': 'What is the current weather in Kandal province, Cambodia?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Kandal province, Cambodia', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Battachi', 'region': 'Kandal', 'country': 'Cambodia', 'lat': 11.45, 'lon': 104.9333, 'tz_id': 'Asia/Phnom_Penh', 'localtime_epoch': 1762677638, 'localtime': '2025-11-09 15:40'}, 'current': {'last_updated_epoch': 1762677000, 'last_updated': '2025-11-09 15:30', 'temp_c': 25.6, 'temp_f': 78.2, 'is_day': 0, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 178, 'wind_dir': 'S', 'pressure_mb': 1009.0, 'pressure_in': 29.8, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 90, 'cloud': 36, 'feelslike_c': 28.9, 'feelslike_f': 84.0, 'windchill_c': 25.6, 'wind

In [3]:
print(state["documents"])

[Document(metadata={}, page_content="{'location': {'name': 'Battachi', 'region': 'Kandal', 'country': 'Cambodia', 'lat': 11.45, 'lon': 104.9333, 'tz_id': 'Asia/Phnom_Penh', 'localtime_epoch': 1762677638, 'localtime': '2025-11-09 15:40'}, 'current': {'last_updated_epoch': 1762677000, 'last_updated': '2025-11-09 15:30', 'temp_c': 25.6, 'temp_f': 78.2, 'is_day': 0, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 178, 'wind_dir': 'S', 'pressure_mb': 1009.0, 'pressure_in': 29.8, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 90, 'cloud': 36, 'feelslike_c': 28.9, 'feelslike_f': 84.0, 'windchill_c': 25.6, 'windchill_f': 78.2, 'heatindex_c': 28.9, 'heatindex_f': 84.0, 'dewpoint_c': 23.9, 'dewpoint_f': 75.0, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 7.3, 'gust_kph': 11.7}}\nWeather now in Kandal Province is 29°C (84°F) and patchy rain possible. Tomorrow it will be 2

In [4]:
# 4. Grade Documents
print("\n--- Step 4: grade_documents ---")
state = rag_node.grade_documents(state)
print("Filtered Documents:")
print_docs(state["documents"])


--- Step 4: grade_documents ---
---CHECK DOUMENTS RELVEVANT TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
Filtered Documents:
Document 1: {'location': {'name': 'Battachi', 'region': 'Kandal', 'country': 'Cambodia', 'lat': 11.45, 'lon': 10 ...


In [5]:
# 5. Decide Next Step
print("\n--- Step 5: decide_to_generatate ---")
next_node = rag_node.decide_to_generate(state)
print("Next Node:", next_node)


--- Step 5: decide_to_generatate ---
---ACCESS THE GRADED DOCUMENT---
---DECISION: GENERATE---
Next Node: generate


In [6]:
# 6. Generate Answer
if next_node == "generate":
    print("\n--- Step 6: generate ---")
    state = rag_node.generate(state)
    print("Generated Answer:\n", state["generation"])
elif next_node == "transform_query":
    print("\n--- Step 6: transform_query again ---")
    state = rag_node.transform_query(state)
    print("New Transformed Question:", state["question"])


--- Step 6: generate ---
---GENERATE---
Generated Answer:
 The current weather in Kandal province, Cambodia is partly cloudy with a temperature of 25.6°C (78.2°F). The humidity is 90% and the wind is blowing at 6.8 km/h from the south. It is not daytime, as indicated by 'is_day' being 0.


In [7]:
# 7. Grade Generation vs Documents and Question
if "generation" in state:
    print("\n--- Step 7: grade_generation_v_documents_and_question ---")
    decision = rag_node.grade_generation_v_documents_and_question(state)
    print("Generation Grading Decision:", decision)

print("\n========== DEBUG SEQUENCE END ==========")


--- Step 7: grade_generation_v_documents_and_question ---
---CHECK HALLUCINATION---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---DECISION: GENERATION ADDRESSES QUESTION---
Generation Grading Decision: accepted

========== DEBUG SEQUENCE END ==========


In [8]:
# 7. Grade Generation vs Documents and Question
print("\n--- Step 7: grade_generation_v_documents_and_question ---")

# Check if generation exists in state
if "generation" not in state:
    print("⚠️ No 'generation' found in state. Did retrieve() produce it?")
else:
    print("Generation exists. Content preview:")
    # Print first 300 chars to avoid flooding console
    print(state["generation"][:300], "..." if len(state["generation"]) > 300 else "")
    
    # Call grading method
    decision = rag_node.grade_generation_v_documents_and_question(state)
    
    # Print grading result
    if decision is None:
        print("⚠️ Grading returned None")
    else:
        print("✅ Generation Grading Decision:", decision)





--- Step 7: grade_generation_v_documents_and_question ---
Generation exists. Content preview:
The current weather in Kandal province, Cambodia is partly cloudy with a temperature of 25.6°C (78.2°F). The humidity is 90% and the wind is blowing at 6.8 km/h from the south. It is not daytime, as indicated by 'is_day' being 0. 
---CHECK HALLUCINATION---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---DECISION: GENERATION ADDRESSES QUESTION---
✅ Generation Grading Decision: accepted


In [1]:
from career_chatbot.data_loader.data_loader import RAGIngestionPipeline

ingestion = RAGIngestionPipeline().run(urls=["https://www.vearochea.com/"])

USER_AGENT environment variable not set, consider setting it to identify your requests.


Splitting 9 documents into chunks...
✅ Split 9 documents into 9 chunks
✅ Stored 9 chunks into Chroma collection 'rag_collection'
